In [20]:
import sys
sys.path.append("..") # This is for the imports from adjacent folders to work

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 200

from main_functions.DCv5_parallel_doubly_stretched import get_circuit as problem_circuit
#from main_functions.DCv5_multiple_XY4 import get_circuit as problem_circuit
from main_functions.gen_dynamic_circuit_v5 import get_circuit as test_circuit
from main_functions.solve_diagonal_ME import solve_diag_ME
from qiskit_aer import AerSimulator
from qiskit import generate_preset_pass_manager
from qiskit.circuit import IfElseOp
from qiskit_ibm_runtime import Batch, SamplerV2
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit_ibm_runtime import QiskitRuntimeService

import os
import pickle
import json
import math
import random
from utils.isa_cdr import generate_isa_clifford_circuits, get_populations, train_model, correct_observable
from utils.get_worst_case_trotter_schedule import get_worst_case_trotter_schedule
from utils.generate_complementary_colors import generate_complementary_colors

In [2]:
QiskitRuntimeService.save_account( 
 token="ryeQLJCCBlp2Y1CK7B0KvXRUvVDG7Xbkhu8H2L9MXUsU", 
 channel="ibm_cloud",
 instance='crn:v1:bluemix:public:quantum-computing:us-east:a/5fef03c739244d07801bbb0cd3b5d388:4854b5b3-784b-48cc-8949-0fe8c86aa28b::', 
 name="quantum_credits",
 overwrite=True
 )

service = QiskitRuntimeService(name="quantum_credits")

In [ ]:
num_emitters = 50
omega_m = [1.2]*num_emitters
omega_c = 1.1
gamma = [0.8*10**(-6)]*num_emitters
kappa = [0.2]
g = [0.03]*(2*(num_emitters-1)) 
t = 200*(np.linspace(0, 1, 5)**1.5)

#initial_state = [str(i) for i in random.sample(range(50), 35)]
# print(initial_state)
initial_state = ['39', '48', '43', '28', '2', '12', '37', '8', '17', '14', '40', '26', '22', '46', '18', '38', '25', '23', '4', '49', '20', '7', '33', '13', '31', '32', '6', '21', '11', '34', '3', '45', '44', '0', '15']

samples = 10**2
noisy_backend = service.backend("ibm_kingston")
optimization_level = 2
shots = 4*(0.5*10**4)
options = {"default_shots": shots}

['39', '48', '43', '28', '2', '12', '37', '8', '17', '14', '40', '26', '22', '46', '18', '38', '25', '23', '4', '49', '20', '7', '33', '13', '31', '32', '6', '21', '11', '34', '3', '45', '44', '0', '15']


In [4]:
k = 1
stored_cliff = []

qc, init_layout = problem_circuit(num_emitters, omega_m, omega_c, g, gamma, kappa, initial_state, k, noisy_backend)
pm = generate_preset_pass_manager(optimization_level, target=noisy_backend.target, initial_layout = init_layout, routing_method = 'none')
isa_qc = pm.run(qc)
pubs = [(isa_qc, x) for x in t]

for x in t:
    clifford_circuits = generate_isa_clifford_circuits(isa_qc, random_init = False, bias = 'inverse', bias_parameter = None, param_value = x, num_variants = samples)
    stored_cliff.append(clifford_circuits)

with open(f"[CCs]50_emitters_{k}_trotter.pkl", "wb") as f:
    pickle.dump(stored_cliff, f)

In [ ]:
with open(f"[CCs]50_emitters_{k}_trotter.pkl", "rb") as f:
    stored_cliff = pickle.load(f)

batch = Batch(backend = noisy_backend, max_time = '1h 20m')
sampler = SamplerV2(mode = batch, options = options)

max_circuits = 1 

main_job_ids = []
cliff_job_ids = []
for idx, instant in enumerate(t):
    main_job = sampler.run([pubs[idx]])
    main_job_ids.append(main_job.job_id())
    print(f'Main job {idx}/{len(t) - 1} submitted with id: {main_job.job_id()}')

    for i in range(0, samples, max_circuits):
        if i + max_circuits <= samples:
            cliff_job = sampler.run(stored_cliff[idx][i : i + max_circuits]) 
        else:
            cliff_job = sampler.run(stored_cliff[idx][i : samples])
        cliff_job_ids.append(cliff_job.job_id())
    
batch.close()

# with open(f"[MJIDs]50_emitters_{k}_trotter.json", "w") as f:
#     json.dump(main_job_ids, f)
# with open(f"[CJIDs]50_emitters_{k}_trotter.json", "w") as f:
#     json.dump(cliff_job_ids, f)

Main job 3/4 submitted with id: d427ie18mjus73cclokg
Main job 4/4 submitted with id: d427jisdgcpc73cutlrg


In [24]:
with open("[MJIDs]50_emitters_1_trotter.json", "r") as f:
    main_job_ids = json.load(f)
with open("[CJIDs]50_emitters_1_trotter.json", "r") as f:
    cliff_job_ids = json.load(f)

main_job_ids

['d41oo6cdgcpc73cueiq0',
 'd41op5h8mjus73cc6q7g',
 'd41oq4sdgcpc73cueld0',
 'd427ie18mjus73cclokg',
 'd427jisdgcpc73cutlrg']